In [ ]:
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1,2"
# os.environ['LD_PRELOAD'] = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4"
sys.path.append('/root/code')

import h5py as h5
from definitions import LOG_DIR, WEIGHT_DIR, DATASET_DIR
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
import time
import datetime
import numpy
from models.Layers import STSA
from models.SCNN18 import SCNN18
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.callbacks import Callback
import utils.dataset as dataset
import logging
from logging import handlers


LOG = logging.getLogger('root')

def initLog(debug=False):
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s %(levelname)s %(message)s',
        datefmt='%Y-%m-%d %H:%M',
        handlers=[logging.StreamHandler(), handlers.RotatingFileHandler('SCNN18_Attention.log', "w", 1024 * 1024 * 100, 3, "utf-8")]
    )
    LOG.setLevel(logging.DEBUG if debug else logging.INFO)
    tf.get_logger().setLevel('ERROR')


initLog()

In [ ]:
def get_optimizer(optimizer, lr):
    optimizer = optimizer.lower()
    if optimizer == 'adadelta':
        return tf.optimizers.Adadelta() if lr == 0 else tf.optimizers.Adadelta(learning_rate=lr)
    elif optimizer == 'adagrad':
        return tf.optimizers.Adagrad() if lr == 0 else tf.optimizers.Adagrad(learning_rate=lr)
    elif optimizer == 'adam':
        return tf.optimizers.Adam() if lr == 0 else tf.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'adamax':
        return tf.optimizers.Adamax() if lr == 0 else tf.optimizers.Adamax(learning_rate=lr)
    elif optimizer == 'sgd':
        return tf.optimizers.SGD() if lr == 0 else tf.optimizers.SGD(learning_rate=lr,momentum=0.9)
    elif optimizer == 'rmsprop':
        return tf.optimizers.RMSprop() if lr == 0 else tf.optimizers.RMSprop(learning_rate=lr)

In [ ]:
# Define hyper-parameter
lr = 1.0
max_epochs = 160
batch_size = 150
input_height = 32000
input_width = 1
nb_classes = 2

In [ ]:
train_ds = dataset.load('./SCNN-Jamendo-train.h5')
# train_ds = train_ds.take(100)
dataset_length = [i for i, _ in enumerate(train_ds)][-1] + 1
# train_ds = train_ds.take(int((dataset_length-1)/4))
print(dataset_length)
train_ds = train_ds.batch(batch_size)
# .shuffle(
    # dataset_length, reshuffle_each_iteration=False)

val_ds = dataset.load('./SCNN-Jamendo-test.h5')
# val_ds =val_ds.take(1)
dataset_length = [i for i, _ in enumerate(val_ds)][-1] + 1
# val_ds = val_ds.take(int((dataset_length-2)/2))
print(dataset_length)
val_ds = val_ds.batch(batch_size)

In [ ]:
class TrainingStepLogger(Callback):
    def on_epoch_end(self, epoch, logs=None):
        LOG.info(f'End of Epoch {epoch + 1} - Loss: {logs["loss"]:.6f}')

In [ ]:
input_shape = (input_height, input_width)

strategy = tf.distribute.MirroredStrategy(devices=[f'/gpu:{i}' for i in range(3)])
with strategy.scope():
    model = SCNN18(input_shape, nb_classes).model()
    optimizer = get_optimizer('adadelta', lr)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary(print_fn=LOG.info)

training_step_logger = TrainingStepLogger()
model.fit(train_ds, validation_data=val_ds,
          epochs=max_epochs,
          callbacks=[training_step_logger])

model.save_weights(os.path.join(WEIGHT_DIR, f"{str(datetime.date.today())}_SCNN18_training_1.h5"))
